In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [11]:
from pandas_datareader import data, wb
import pandas_datareader as pdr

In [25]:
start = pd.to_datetime('2017-01-01')
end = pd.to_datetime('2017-07-01')
df = pdr.get_data_yahoo('AAPL', start = start, end = end)

In [27]:
df['Change'] = df.Close - df.Open
df.head()

,Open,High,Low,Close,Adj Close,Volume,Change
Date,,,,,,,
2017-01-03,115.800003,116.330002,114.760002,116.150002,115.173210,28781900,0.349999
2017-01-04,115.849998,116.510002,115.750000,116.019997,115.044304,21118100,0.169999
2017-01-05,115.919998,116.860001,115.809998,116.610001,115.629341,22193600,0.690003
2017-01-06,116.779999,118.160004,116.470001,117.910004,116.918411,31751900,1.130005
2017-01-09,117.949997,119.430000,117.940002,118.989998,117.989319,33561900,1.040001


In [54]:
changemax = df.Change.max()
changemin = df.Change.min()
bins = [changemin-0.001, 0, changemax+0.001]
labels = ['Down', 'Up']
df['Trend'] = pd.cut(df.Change, bins=bins, labels = labels).astype('category')
df['Activity'] = pd.cut(df.Volume, bins=[df.Volume.min()-1,df.Volume.mean(),df.Volume.max()],
                        labels = ['low','high']).astype('category')
df.head()

,Open,High,Low,Close,Adj Close,Volume,Change,Trend,Activity
Date,,,,,,,,,
2017-01-03,115.800003,116.330002,114.760002,116.150002,115.173210,28781900,0.349999,Up,high
2017-01-04,115.849998,116.510002,115.750000,116.019997,115.044304,21118100,0.169999,Up,low
2017-01-05,115.919998,116.860001,115.809998,116.610001,115.629341,22193600,0.690003,Up,low
2017-01-06,116.779999,118.160004,116.470001,117.910004,116.918411,31751900,1.130005,Up,high
2017-01-09,117.949997,119.430000,117.940002,118.989998,117.989319,33561900,1.040001,Up,high


### advanced groupby

In [56]:
df.groupby(['Trend','Activity'])[['Change','Close']].mean()

Change       Close
Trend Activity                      
Down  low      -0.463334  142.936888
      high     -1.472145  142.203571
Up    low       0.605897  137.617948
      high      1.368518  136.748888

In [57]:
df.groupby(['Trend','Activity']).agg({'Change':[np.mean, np.std],'Close':[np.mean, np.std]})

Close               Change          
                      mean        std      mean       std
Trend Activity                                           
Down  low       142.936888   8.671730 -0.463334  0.434568
      high      142.203571   7.500006 -1.472145  1.682998
Up    low       137.617948  11.529426  0.605897  0.464621
      high      136.748888  13.547194  1.368518  0.861691

#### apply function to groupby iterator and return original dataframe format

In [58]:
def averageChange(dataframe):
    dataframe['averageChange'] = dataframe.Change.mean()
    return dataframe

In [59]:
newdf = df.groupby('Trend').apply(averageChange)

In [62]:
newdf.head()

,Open,High,Low,Close,Adj Close,Volume,Change,Trend,Activity,averageChange
Date,,,,,,,,,,
2017-01-03,115.800003,116.330002,114.760002,116.150002,115.173210,28781900,0.349999,Up,high,0.917878
2017-01-04,115.849998,116.510002,115.750000,116.019997,115.044304,21118100,0.169999,Up,low,0.917878
2017-01-05,115.919998,116.860001,115.809998,116.610001,115.629341,22193600,0.690003,Up,low,0.917878
2017-01-06,116.779999,118.160004,116.470001,117.910004,116.918411,31751900,1.130005,Up,high,0.917878
2017-01-09,117.949997,119.430000,117.940002,118.989998,117.989319,33561900,1.040001,Up,high,0.917878


#### apply function to groupby iterator and return grouped dataframe

In [67]:
def ChangeRank(dataframe, col):
    dataframe.sort_values(by=col, ascending=False, inplace=True)
    dataframe[col + 'Rank'] = np.arange(len(dataframe))+1
    return dataframe

In [68]:
dfranked = df.groupby(['Trend','Activity']).apply(lambda x: ChangeRank(x,'Change'))

In [71]:
dfranked[dfranked.ChangeRank==1]

Open        High         Low       Close  \
Trend Activity Date                                                         
Down  low      2017-05-22  154.000000  154.580002  152.910004  153.990005   
      high     2017-02-15  135.520004  136.270004  134.619995  135.509995   
Up    low      2017-03-22  139.850006  141.600006  139.759995  141.419998   
      high     2017-05-08  149.029999  153.699997  149.029999  153.009995   

                            Adj Close    Volume    Change Trend Activity  \
Trend Activity Date                                                        
Down  low      2017-05-22  153.990005  22966400 -0.009995  Down      low   
      high     2017-02-15  134.952957  35623100 -0.010009  Down     high   
Up    low      2017-03-22  140.838669  25860200  1.569992    Up      low   
      high     2017-05-08  152.381027  48752400  3.979996    Up     high   

                           ChangeRank  
Trend Activity Date                    
Down  low      2017-05-22           1  
      high     2017-02-15           1  
Up    low      2017-03-22           1  
      high     2017-05-08           1

#### transform

In [72]:
zscore = lambda x: (x - x.mean())/x.std()

In [73]:
df.groupby('Trend').Change.transform(zscore)

Date
2017-01-03   -0.754853
2017-01-04   -0.994118
2017-01-05   -0.302903
2017-01-06    0.281970
2017-01-09    0.162332
2017-01-10   -0.768139
2017-01-11    0.122456
2017-01-12   -0.754854
2017-01-13    0.643921
2017-01-17    0.986471
2017-01-18    0.704981
2017-01-19   -0.714978
2017-01-20    0.257193
2017-01-23   -1.113746
2017-01-24   -0.661807
2017-01-25    0.720614
2017-01-26   -0.861186
2017-01-27    0.521793
2017-01-30   -0.289618
2017-01-31   -0.954244
2017-02-01    1.066221
2017-02-02   -0.489007
2017-02-03   -0.196562
2017-02-06    0.321824
2017-02-07    0.095876
2017-02-08   -0.302924
2017-02-09   -0.196562
2017-02-10    0.369126
2017-02-13   -0.940959
2017-02-14    0.840252
                ...   
2017-05-19    0.389485
2017-05-22    0.704988
2017-05-23   -0.404315
2017-05-24    0.206304
2017-05-25   -1.033995
2017-05-26    0.318253
2017-05-30   -0.887777
2017-05-31   -0.516279
2017-06-01   -1.206803
2017-06-02    1.265601
2017-06-05    0.297895
2017-06-06   -0.488998
2017-0